<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Working modules

In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

## Подготовка данных

In [9]:
df = pd.read_csv(r'E:\COURSES\Кодинг\[Yandex praxis]. DS specialist + (2022)\6\Часть 6\07 Самостоятельный проект\churn.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [11]:
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0


In [12]:
df['Tenure'] = df['Tenure'].fillna(0)

In [13]:
df.Surname.nunique()

2932

In [14]:
df.Geography.nunique()

3

In [15]:
df.duplicated().sum()

0

In [16]:
data_ohe = (pd
    .get_dummies(
    df,
    columns=['Geography'],
    drop_first=True,
    dtype=int)
    )
data_ohe = (pd
            .get_dummies(
    data_ohe,
    columns=['Gender'],
    drop_first=True,
    dtype=int)
)


In [17]:
data_ohe.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,1,15634602,Hargrave,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,2,15647311,Hill,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0


In [18]:
data_ohe.Gender_Male.value_counts()

Gender_Male
1    5457
0    4543
Name: count, dtype: int64

### Масштабирование признаков

In [19]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
data_ohe[numeric] = scaler.fit_transform(data_ohe[numeric])

## Исследование задачи

In [20]:
features = data_ohe.drop(['Exited','Surname','RowNumber', 'CustomerId'],axis=1)
target = data_ohe['Exited']
features_train, features_valid, target_train, target_valid = train_test_split(features, target,test_size=0.25,random_state=12345)

In [21]:
features.head(2)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,-0.326221,0.293517,-0.817441,-1.225848,-0.911583,1,1,0.021886,0,0,0
1,-0.440036,0.198164,-1.138838,0.117350,-0.911583,0,1,0.216534,0,1,0


In [22]:
target.value_counts(normalize=1)

Exited
0    0.7963
1    0.2037
Name: proportion, dtype: float64

## 1 DecisionTree

In [23]:
param_grid_dt = {
    'criterion':['gini','entropy'],
    'max_depth':range(1,15,1),
    'min_samples_leaf':range(1,5,1),
    'min_samples_split':range(2,10,1),
    'max_features':['None','sqrt','log2']
}
grid_search_dt = GridSearchCV(estimator=DecisionTreeClassifier(),
                           param_grid=param_grid_dt,
                           cv=5,
                           scoring='f1',
                           n_jobs=-1)

In [24]:
grid_search_dt.fit(features_train, target_train)
print("Best Parameters:", grid_search_dt.best_params_)

Best Parameters: {'criterion': 'gini', 'max_depth': 9, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 3}


C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
4480 fits failed out of a total of 13440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2369 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-p

In [25]:
best_model_dt = grid_search_dt.best_estimator_
predicted_valid_dt = best_model_dt.predict(features_valid)
f1_dt = f1_score(target_valid,predicted_valid_dt)
dtc_share = pd.Series(predicted_valid_dt).value_counts(normalize=True)
print(dtc_share)
print("F1 score:",f1_dt)

0    0.8924
1    0.1076
Name: proportion, dtype: float64
F1 score: 0.527363184079602


## 2. RandomForest

In [49]:
param_grid_rf = {
    'n_estimators':range(80,120,10),
    'criterion':['gini','entropy'],
    'max_depth':range(1,11,2),
    'min_samples_leaf':range(1,2,4),
    'min_samples_split':range(2,5,10),
    'max_features':['None','sqrt','log2'],
    'bootstrap': [True, False],
    'class_weight': ['balanced', 'balanced_subsample'],
}
grid_search_rf = GridSearchCV(estimator=RandomForestClassifier(),
                              param_grid=param_grid_rf,
                              cv=5,
                              scoring='f1',
                              n_jobs=-1,
                              verbose=1)

In [50]:
grid_search_rf.fit(features_train, target_train)
print("Best Parameters:", grid_search_rf.best_params_)

Fitting 5 folds for each of 480 candidates, totalling 2400 fits


C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
800 fits failed out of a total of 2400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
499 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-pack

Best Parameters: {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 9, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 110}


In [28]:
best_model_rf = grid_search_rf.best_estimator_
predicted_valid_rf = best_model_rf.predict(features_valid)
f1_rf = f1_score(target_valid, predicted_valid_rf)
rf_share = pd.Series(predicted_valid_rf).value_counts(normalize=True)
print(rf_share)
print("F1 score:", f1_rf)

0    0.888
1    0.112
Name: proportion, dtype: float64
F1 score: 0.5595092024539877


### 3. Logistic Regression

In [46]:
param_grid_lr = [
    {
        'penalty': ['l1', 'l2'],
        'C': np.logspace(-4, 4, 10),
        'solver': ['liblinear', 'saga'],  # 'liblinear' works for l1/l2
        'class_weight': [None, 'balanced']
    },
    {
        'penalty': ['elasticnet'],
        'C': np.logspace(-4, 4, 10),
        'solver': ['saga'],  # Only 'saga' supports elasticnet
        'l1_ratio': [0.1, 0.5, 0.9],  # Required for elasticnet
        'class_weight': [None, 'balanced']
    },
    {
        'penalty': [None],
        'solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],  # No penalty
        'class_weight': [None, 'balanced']
    }
]

grid_search_lr = GridSearchCV(estimator=LogisticRegression(),
                              param_grid=param_grid_lr,
                              cv=5,
                              scoring='f1',
                              n_jobs=-1,
                              verbose = 1)

In [47]:
grid_search_lr.fit(features_train, target_train)
print("Best Parameters:", grid_search_lr.best_params_)

Fitting 5 folds for each of 148 candidates, totalling 740 fits
Best Parameters: {'C': 0.005994842503189409, 'class_weight': 'balanced', 'l1_ratio': 0.9, 'penalty': 'elasticnet', 'solver': 'saga'}


In [48]:
print("Best Parameters:", grid_search_lr.best_params_)
best_model_lr = grid_search_lr.best_estimator_
predicted_valid_lr = best_model_lr.predict(features_valid)
f1_lr = f1_score(target_valid, predicted_valid_lr)
lr_share = pd.Series(predicted_valid_lr).value_counts(normalize=True)
print(lr_share)
print("F1 score:", f1_lr)

Best Parameters: {'C': 0.005994842503189409, 'class_weight': 'balanced', 'l1_ratio': 0.9, 'penalty': 'elasticnet', 'solver': 'saga'}
0    0.624
1    0.376
Name: proportion, dtype: float64
F1 score: 0.5193220338983051


## Борьба с дисбалансом

How we can see all models are either higly above the target ratio of exited or below

In [32]:
predicted_valid_c = pd.Series([0]*len(target_valid))
f1_c = f1_score(target_valid, predicted_valid_c)
print('overall score',accuracy_score(target_valid,predicted_valid_c))
print("F1 score:", f1_c)

overall score 0.786
F1 score: 0.0


### Confusion matrix

In [33]:
confusion_matrix_dt =  confusion_matrix(target_valid, predicted_valid_dt)
confusion_matrix_lr =  confusion_matrix(target_valid, predicted_valid_lr)
confusion_matrix_rf =  confusion_matrix(target_valid, predicted_valid_rf)
print('confusion_matrix_rf:',confusion_matrix_rf)
print('confusion_matrix_lr:',confusion_matrix_lr)
print('confusion_matrix_dt:',confusion_matrix_dt)

confusion_matrix_rf: [[1913   52]
 [ 307  228]]
confusion_matrix_lr: [[1408  557]
 [ 152  383]]
confusion_matrix_dt: [[1908   57]
 [ 323  212]]


### Balancing samples

#### Upsample

In [34]:
target_train.value_counts(normalize=True)

Exited
0    0.799733
1    0.200267
Name: proportion, dtype: float64

In [35]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=333)
    return features_upsampled, target_upsampled


def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat([features_ones] + [features_zeros.sample(frac=fraction, random_state=333)])
    target_downsampled = pd.concat([target_ones] + [target_zeros.sample(frac=fraction, random_state=333)])
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=333)
    return features_downsampled, target_downsampled

In [45]:
repeat_values = np.arange(1, 5, 1)
fraction_values = np.arange(0.1, 1.1, 0.1)
f1_sampling_best = 0
repeat_best = None
fraction_best = None
for repeat in repeat_values:
    for fraction in fraction_values:
        features_upsampled, target_upsampled = upsample(features, target, repeat)
        features_resampled, target_resampled = downsample(features_upsampled, target_upsampled, fraction)
        grid_search_lr = GridSearchCV(estimator=LogisticRegression(),
                                      param_grid=param_grid_lr,
                                      cv=5,
                                      scoring='f1',
                                      n_jobs=-1)
        grid_search_lr.fit(features_resampled, target_resampled)
        best_model_lr = grid_search_lr.best_estimator_
        predicted_valid_lr = best_model_lr.predict(features)
        f1_lr = f1_score(predicted_valid_lr, target)
        if f1_lr > f1_sampling_best:
            f1_sampling_best = f1_lr
            grid_search_lr = grid_search_lr.best_estimator_
            fraction_best = fraction
            repeat_best = repeat

print('fraction best', fraction_best, 'repeat best', repeat_best)

KeyboardInterrupt: 

In [37]:
print(f1_sampling_best)

0.5065236818588025


In [38]:
features_upsampled, target_upsampled = upsample(features, target, repeat_best)
features_downsampled, target_downsampled = downsample(features_upsampled, target_upsampled, fraction_best)
features_resampled_train, target_resampled_train, features_resampled_valid, target_resampled_valid = train_test_split(
    features_downsampled, target_downsampled, test_size=0.25, random_state=333)

#### Same on decision tree

In [42]:
grid_search_dt.fit(features_downsampled, target_downsampled)
print("Best Parameters:", grid_search_dt.best_params_)

best_model_dt = grid_search_dt.best_estimator_
predicted_valid_dt = best_model_dt.predict(features_valid)
f1_dt = f1_score(target_valid, predicted_valid_dt)
dtc_share = pd.Series(predicted_valid_dt).value_counts(normalize=True)
print(dtc_share)
print("F1 score:", f1_dt)

Best Parameters: {'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 6}
0    0.7744
1    0.2256
Name: proportion, dtype: float64
F1 score: 0.6132848043676069


C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
4480 fits failed out of a total of 13440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1740 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-p

#### Same on Random Forest

In [41]:
grid_search_rf.fit(features_downsampled, target_downsampled)
print("Best Parameters:", grid_search_rf.best_params_)

best_model_rf = grid_search_rf.best_estimator_
predicted_valid_rf = best_model_rf.predict(features_valid)
f1_rf = f1_score(target_valid, predicted_valid_rf)
rf_share = pd.Series(predicted_valid_rf).value_counts(normalize=True)
print(rf_share)
print("F1 score:", f1_rf)

C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
200 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
134 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\Александр\AppData\Local\Programs\Python\Python312\Lib\site-packa

Best Parameters: {'criterion': 'entropy', 'max_depth': 9, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 110}
0    0.7772
1    0.2228
Name: proportion, dtype: float64
F1 score: 0.7252747252747253


b## Тестирование модели

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: задача исследована
    - [ ]  Исследован баланс классов
    - [ ]  Изучены модели без учёта дисбаланса
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 3: учтён дисбаланс
    - [ ]  Применено несколько способов борьбы с дисбалансом
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 4: проведено тестирование
- [ ]  Удалось достичь *F1*-меры не менее 0.59
- [ ]  Исследована метрика *AUC-ROC*